In [28]:
import sys
sys.path.append('/oak/stanford/groups/dpwall/computeEnvironments/micropheno/MicroPheno/')
from make_representations.representation_maker import Metagenomic16SRepresentation, FastaRepresentations
import numpy as np
import pandas as pd
from Bio import SeqIO
import scipy.sparse
from tqdm import tqdm
BIOMARKER_DIR = '/home/groups/dpwall/briannac/sequence_based_biomarkers/results/generate_biomarkers/'

In [29]:
def sparse_unique_columns(M):
    M = M.tocsc()
    m, n = M.shape
    if not M.has_sorted_indices:
        M.sort_indices()
    if not M.has_canonical_format:
        M.sum_duplicates()
    sizes = np.diff(M.indptr)
    idx = np.argsort(sizes)
    Ms = M@scipy.sparse.csc_matrix((np.ones((n,)), idx, np.arange(n+1)), (n, n))
    ssizes = np.diff(Ms.indptr)
    ssizes[1:] -= ssizes[:-1]
    grpidx, = np.where(ssizes)
    grpidx = np.concatenate([grpidx, [n]])
    if ssizes[0] == 0:
        counts = [np.array([0, grpidx[0]])]
    else:
        counts = [np.zeros((1,), int)]
    ssizes = ssizes[grpidx[:-1]].cumsum()
    for i, ss in tqdm(enumerate(ssizes)):
        gil, gir = grpidx[i:i+2]
        pl, pr = Ms.indptr[[gil, gir]]
        dv = Ms.data[pl:pr].view(f'V{ss*Ms.data.dtype.itemsize}')
        iv = Ms.indices[pl:pr].view(f'V{ss*Ms.indices.dtype.itemsize}')
        idxi = np.lexsort((dv, iv))
        dv = dv[idxi]
        iv = iv[idxi]
        chng, = np.where(np.concatenate(
            [[True], (dv[1:] != dv[:-1]) | (iv[1:] != iv[:-1]), [True]]))
        counts.append(np.diff(chng))
        idx[gil:gir] = idx[gil:gir][idxi]
    counts = np.concatenate(counts)
    nu = counts.size - 1
    uniques = M@scipy.sparse.csc_matrix((np.ones((nu,)), idx[counts[:-1].cumsum()],
                                   np.arange(nu + 1)), (n, nu))
    return uniques, idx[counts[:-1].cumsum()], counts[1:]


In [30]:
# Compute person x biomarker matrix
def SaveMatrices(asv_biomarker, biomarkers, dataset, biomarker_type):
    sample_vs_asv = pd.read_table('/home/groups/dpwall/briannac/sequence_based_biomarkers/data/%s/sample_vs_asv.tsv' % dataset, index_col=0)
    person_biomarker = scipy.sparse.csc_matrix(sample_vs_asv.transpose())*asv_biomarker
    biomarker_exists  = [True for _ in biomarkers]
        
    _, no_ld_idx, _ = sparse_unique_columns(asv_biomarker)
    no_ld_idx = sorted(no_ld_idx)

    print("Number of biomarkers after LD filtering: ", len(no_ld_idx))

        #np.save('%sperson_variant%d_condensed' %  (output_dir,order), person_biomarker)
    print('Computing biomarker names...')

        #asv_biomarker = scipy.sparse.csr_matrix(asv_biomarker[:,no_ld_idx])
    person_biomarker = scipy.sparse.csr_matrix(person_biomarker[:,no_ld_idx])

    is_abundant = ((person_biomarker>0).mean(axis=0)>.1).tolist()[0]
    print(sum(is_abundant), ' biomarkers > 10% freq')
    asv_biomarker = asv_biomarker[:,np.where(is_abundant)[0]]
    person_biomarker = person_biomarker[:,np.where(is_abundant)[0]]

    unique_idx_current = 0
    current_idx = 0
    with open(BIOMARKER_DIR + 'biomarker_names_%s_%s.txt' % (biomarker_type, dataset), 'w') as f:
        for i,biomarker in tqdm(enumerate(biomarkers)):
            if biomarker_exists[i]:  # If biomarker doesn't even exist, just skip.
                if unique_idx_current==len(no_ld_idx):
                    break
                elif current_idx==no_ld_idx[unique_idx_current]:
                    if is_abundant[unique_idx_current]: 
                        f.write(str(biomarker) + '\n')
                    unique_idx_current += 1
                current_idx += 1
    print("Writing to files...")
    scipy.sparse.save_npz(BIOMARKER_DIR + 'asv_vs_biomarker_%s_%s.npz' % (biomarker_type, dataset), asv_biomarker)  # Save sparse matrix of ASV x biomarker.
    scipy.sparse.save_npz(BIOMARKER_DIR + 'sample_vs_biomarker_%s_%s.npz' % (biomarker_type, dataset), person_biomarker)  # Save sparse matrix of ASV x biomarker.


# Micropheno

In [31]:
for dataset in ['autism', 'obesity']:
    for k in [4,6,8]:
        seqs_file='/home/groups/dpwall/briannac/sequence_based_biomarkers/data/%s/seqs.fa' % dataset
        
        FastaRep=FastaRepresentations(seqs_file,label_modifying_func=lambda x: x.split('.')[0])
        asv_biomarker=FastaRep.get_vector_rep(FastaRep.corpus, k,restricted=True)
        
        SaveMatrices(asv_biomarker, [i for i in range(np.shape(asv_biomarker)[1])], dataset, 'micropheno%s' % k)

244it [00:00, 26012.51it/s]
256it [00:00, 428981.95it/s]


Number of biomarkers after LD filtering:  256
Computing biomarker names...
256  biomarkers > 10% freq
Writing to files...


832it [00:00, 27292.19it/s]
4096it [00:00, 511016.66it/s]


Number of biomarkers after LD filtering:  4053
Computing biomarker names...
3595  biomarkers > 10% freq
Writing to files...


775it [00:00, 21436.20it/s]
65536it [00:00, 818807.90it/s]

...

Number of biomarkers after LD filtering:  256
Computing biomarker names...
256  biomarkers > 10% freq
Writing to files...


904it [00:00, 27181.85it/s]
4096it [00:00, 543752.78it/s]


Number of biomarkers after LD filtering:  3962
Computing biomarker names...
2855  biomarkers > 10% freq
Writing to files...


904it [00:00, 22346.02it/s]
0it [00:00, ?it/s]


Number of biomarkers after LD filtering:  13350
Computing biomarker names...
4878  biomarkers > 10% freq
Writing to files...


# 1 SBB

In [32]:
for dataset in ['autism', 'obesity']:
    rdp = [record for record in SeqIO.parse('/home/groups/dpwall/briannac/sequence_based_biomarkers/data/%s/seqs_aligned_rdp.fa' % dataset, 'fasta')][:-1] 
    df = pd.DataFrame([list(r.seq.upper()) for r in rdp])
    is_interesting = df.apply(lambda x: (len(set(x))>0) & (len(set(x).intersection(['A', 'T', 'C', 'G']))>0))

    df = df[df.columns[is_interesting]]
    df_A = df=='A'
    df_A.columns = [str(c) + '_A' for c in df_A.columns]
    df_T = df=='T'
    df_T.columns = [str(c) + '_T' for c in df_T.columns]
    df_G = df=='G'
    df_G.columns = [str(c) + '_G' for c in df_G.columns]
    df_C = df=='C'
    df_C.columns = [str(c) + '_C' for c in df_C.columns]
    df_gap = df=='-'
    df_gap.columns = [str(c) + '_gap' for c in df_gap.columns]
    df_sbb1 = pd.concat([df_A, df_C, df_T, df_G, df_gap], axis=1)
    df_sbb1 = df_sbb1[df_sbb1.columns[(df_sbb1.mean()>0.0) & (df_sbb1.mean()<1.0)]]
    df_sbb1 = df_sbb1[df_sbb1.columns[~df_sbb1.transpose().duplicated().values]]

    df_sbb1 = pd.concat([df_A, df_C, df_T, df_G, df_gap], axis=1)

    # Get rid of biomarkers at 100% frequency, 0% frequency, and in LD w/another.
    df_sbb1 = df_sbb1[df_sbb1.columns[(df_sbb1.mean()>0.0) & (df_sbb1.mean()<1.0)]]
    df_sbb1 = df_sbb1[df_sbb1.columns[~df_sbb1.transpose().duplicated().values]]

    # Save sparse matrix of ASV x biomarker.
    asv_biomarker = scipy.sparse.csc_matrix(df_sbb1)
    SaveMatrices(asv_biomarker, df_sbb1.columns, dataset, 'sbb1')

488it [00:00, 26149.76it/s]
1121it [00:00, 527997.17it/s]


Number of biomarkers after LD filtering:  1121
Computing biomarker names...
644  biomarkers > 10% freq
Writing to files...


498it [00:00, 25084.83it/s]
912it [00:00, 508603.28it/s]


Number of biomarkers after LD filtering:  912
Computing biomarker names...
594  biomarkers > 10% freq
Writing to files...


# 1 SBB for testing

In [33]:
for dataset in ['autism', 'obesity']:
    rdp = [record for record in SeqIO.parse('/home/groups/dpwall/briannac/sequence_based_biomarkers/data/%s/seqs_aligned_rdp.fa' % dataset, 'fasta')][:-1] 
    df = pd.DataFrame([list(r.seq.upper()) for r in rdp])
    is_interesting = df.apply(lambda x: (len(set(x))>0) & (len(set(x).intersection(['A', 'T', 'C', 'G']))>0))

    df = df[df.columns[is_interesting]]
    df_A = df=='A'
    df_A.columns = [str(c) + '_A' for c in df_A.columns]
    df_T = df=='T'
    df_T.columns = [str(c) + '_T' for c in df_T.columns]
    df_G = df=='G'
    df_G.columns = [str(c) + '_G' for c in df_G.columns]
    df_C = df=='C'
    df_C.columns = [str(c) + '_C' for c in df_C.columns]
    df_gap = df=='-'
    df_gap.columns = [str(c) + '_gap' for c in df_gap.columns]
    df_sbb1 = pd.concat([df_A, df_C, df_T, df_G, df_gap], axis=1)
    df_sbb1 = df_sbb1[df_sbb1.columns[(df_sbb1.mean()>0.0) & (df_sbb1.mean()<1.0)]]
    df_sbb1 = df_sbb1[df_sbb1.columns[~df_sbb1.transpose().duplicated().values]]
    df_sbb1 = pd.concat([df_A, df_C, df_T, df_G, df_gap], axis=1)

    # Get rid of biomarkers at 100% frequency, 0% frequency, and in LD w/another.
    df_sbb1 = df_sbb1[df_sbb1.columns[(df_sbb1.mean()>0.0) & (df_sbb1.mean()<1.0)]]
    df_sbb1 = df_sbb1[df_sbb1.columns[~df_sbb1.transpose().duplicated().values]]
    df_sbb1 = df_sbb1[df_sbb1.columns[::20]] # JUST 100 COLUMNS FOR TESTING
    print(np.shape(df_sbb1))
    # Save sparse matrix of ASV x biomarker.
    asv_biomarker = scipy.sparse.csc_matrix(df_sbb1)
    SaveMatrices(asv_biomarker, df_sbb1.columns, dataset, 'sbb1')

(5249, 57)


43it [00:00, 26002.75it/s]
57it [00:00, 275750.09it/s]


Number of biomarkers after LD filtering:  57
Computing biomarker names...
34  biomarkers > 10% freq
Writing to files...
(12363, 46)


44it [00:00, 25451.58it/s]
46it [00:00, 245780.87it/s]

Number of biomarkers after LD filtering:  46
Computing biomarker names...
37  biomarkers > 10% freq
Writing to files...


# OTUs

In [34]:
for dataset in ['autism', 'obesity']:
    for similarity in [90, 95, 97, 99]:
        otu_mapping = pd.read_table('/home/groups/dpwall/briannac/sequence_based_biomarkers/results/generate_biomarkers/otu_asv_map_%i_%s.txt' % (similarity, dataset), header=None)
        otu_dict = {otu:idx for idx, otu in enumerate(np.unique(otu_mapping[1]))}
        otu_mapping[1] = [otu_dict[i] for i in otu_mapping[1]]
        mat = np.zeros((len(otu_mapping), len(otu_dict)))
        for i, otu_id in enumerate(otu_mapping[1]):
            mat[i][otu_id] = 1
        asv_biomarker = scipy.sparse.csc_matrix(mat)
        SaveMatrices(asv_biomarker, otu_dict.keys, dataset, 'otu%s' % similarity)

TypeError: 'builtin_function_or_method' object is not iterable

# ASVs

In [ ]:
for dataset in ['autism', 'obesity']:
    sample_vs_asv = pd.read_table('/home/groups/dpwall/briannac/sequence_based_biomarkers/data/%s/sample_vs_asv.tsv' % dataset, index_col=0)

    # Save sparse matrix of ASV x biomarker.
    asv_biomarker = scipy.sparse.csc_matrix(np.identity(len(sample_vs_asv)))
    SaveMatrices(asv_biomarker, sample_vs_asv.index, dataset, 'asv')

# Taxa

In [ ]:
for dataset in ['autism', 'obesity']:
    annotations = pd.read_table('/home/groups/dpwall/briannac/sequence_based_biomarkers/data/%s/asv_vs_taxa_annotation.tsv' % dataset, index_col=0)
    sample_vs_asv = pd.read_table('/home/groups/dpwall/briannac/sequence_based_biomarkers/data/%s/sample_vs_asv.tsv' % dataset, index_col=0)
    annotations['Kingdom'] = ['k_' +a for a in annotations['Kingdom']]
    annotations['Phylum'] = ['p_' +a for a in annotations['Phylum']]
    annotations['Class'] = ['c_' +a for a in annotations['Class']]
    annotations['Order'] = ['o_' +a for a in annotations['Order']]
    annotations['Family'] = ['f_' +a for a in annotations['Family']]
    annotations['Genus'] = ['g_' +a for a in annotations['Genus']]
    taxa = np.unique(np.concatenate(annotations.values))
    taxa_df = pd.DataFrame(np.zeros((len(annotations), len(taxa))))
    taxa_df.columns = taxa
    taxa_df.index = annotations.index
    taxa_df = taxa_df[[c for c in taxa_df.columns if 'unclassified' not in c]]
    for row in annotations.iterrows():
        taxa_df.loc[row[0],list([r for r in row[1].values if 'unclassified' not in r])]  = 1
        
        
    # Save sparse matrix of ASV x biomarker.
    asv_biomarker = scipy.sparse.csc_matrix(taxa_df)
    SaveMatrices(asv_biomarker, taxa_df.columns, dataset, 'taxa')